# Setup

In [2]:
# Import packages we'll need later on in this tutorial
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Import BW25 packages. You'll notice the packages are imported individually, unlike a one-and-done import with BW2.
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as bwa

In [3]:
bd.projects

Brightway2 projects manager with 2 objects:
	autumn-school
	default
Use `projects.report()` to get a report on all projects.

In [4]:
# You need to set a project. Give it a name!
name = "autumn-school"
bd.projects.set_current(name)

In [5]:
# If this is your first time using this BW25 project, this should be an empty dictionary! 
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent-391-cutoff

In [6]:
# If this is the first time you've set up the project, you need to install the biosphere flows.
# You do not need to run this subsequent times.

bi.bw2setup() ## THIS YIELDS AN ERROR AT THE MOMENT

Biosphere database already present!!! No setup is needed


In [7]:
# Import the path where your EI database is stored. 
# Note that the EI database must be unzipped and the path should end at the datasets folder.
ei_path = r"C:\Users\pierr\cutoff39\datasets"

# You will also need to give your database a name. This name will appear when you call bd.databases.
# Here, I am using EI v3.9.1 cutoff.
ei_name = "ecoinvent-391-cutoff"

# When we execute this cell, we will check if it's already been imported, and if not (else) we import it.

if ei_name in bd.databases:
    print("Database has already been imported.")
else:
# Go ahead and import:
    ei_importer = bi.SingleOutputEcospold2Importer(ei_path, ei_name)
    # Apply stragegies 
    ei_importer.apply_strategies()
    # We can get some statistics
    ei_importer.statistics()
    # Now we will write the database into our project. 
    ei_importer.write_database()

Database has already been imported.


In [8]:
ei=bd.Database("ecoinvent-391-cutoff")

In [9]:
print([act for act in ei if "steel production, electric, low-alloyed" == act['name'] and "Europe without Switzerland and Austria" == act['location']])

['steel production, electric, low-alloyed' (kilogram, Europe without Switzerland and Austria, None)]


In [10]:
activity=[act for act in ei if "transport, freight, lorry 16-32 metric ton, EURO6" == act['name'] and "RER" == act['location']][0]
activity.as_dict()['reference product']

'transport, freight, lorry 16-32 metric ton, EURO6'

In [11]:
activity.as_dict()['reference product']

'transport, freight, lorry 16-32 metric ton, EURO6'

In [12]:
bi.CSVImporter?

Init signature: bi.CSVImporter(filepath)
Docstring:      Generic CSV importer
Init docstring:
Initialize the ImportBase object.

Parameters
----------
*args :
    Variable length argument list.
**kwargs :
    Arbitrary keyword arguments.

Raises
------
NotImplemented :
    This class should be subclassed.
File:           c:\users\pierr\anaconda3\envs\bw\lib\site-packages\bw2io\importers\excel.py
Type:           type
Subclasses:     

In [13]:
i=bi.CSVImporter(r"C:/Users/pierr/autumn-school-bw2/lci-reinforcing-steel.csv")
i.apply_strategies()
i.match_database(fields=["name", "location", "unit"])
i.match_database("ecoinvent-391-cutoff", fields=["name", "reference product", "unit", "location"])
i.match_database("biosphere3", fields=["name", "categories"])
#i.statistics()
#i.drop_unlinked(i_am_reckless=True)
i.write_database()

Extracted 1 worksheets in 0.00 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 29.74 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Not able to determine geocollections for al

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


InvalidExchange: 

In [ ]:
activity=[act for act in ei if "reinforcing steel production" == act['name'] and "RER" == act['location']][0]